## Introduction to Dropconnect


Instead of deactivating entire neurons (Dropout), DropConnect sets individual weights to 0 (i.e., deactivates them). Since neurons consist of multiple weights, one could say that the neurons are only partially deactivated.

DropConnect is a regularization technique for neural networks that works similarly to Dropout, but instead of deactivating entire neurons, it randomly disables individual weight connections.

## Dropout

Drop

```text
[N1]---\
[N2]----X <- dropped out (entire neuron)
[N3]---/

## Dropconnect

```text
[N1]-X--\
[N2]--X--X <- some individual connections are missing
[N3]-----/

In the practical implementation of DropConnect in Probly, entire layers (neurons) are considered. To set specific weights to 0, whole layers must be modified and replaced with new ones.

**Imports**

In [13]:
from probly.transformation import dropconnect
import torch.nn as nn
from flax import nnx
import jax
from typing import Any, cast

Calling the model:

In [14]:
# Defining a basic model
model = nn.Sequential(
    nn.Linear(2, 2),
    nn.Linear(2, 2)
)

# Applying DropConnect to the model
model_dc = dropconnect(model, p=0.5)

# Testing DropConnect
assert isinstance(model_dc[0], nn.Linear)
assert hasattr(model_dc[1], "p")
assert model_dc[1].p == 0.5


The variable p is the dropout probability for the weights.

The variable flax_model_small_2d_2d (or flax_conv_linear_model) is the model that will be processed and modified by DropConnect.

flax_custom_model:

In [15]:
# Create RNGs
rngs = nnx.Rngs(params=jax.random.key(0))

# Define a custom Flax model
class FlaxModel(nnx.Module):
    def __init__(self, rngs):
        self.fc1 = nnx.Linear(2, 2, rngs=rngs)
        self.fc2 = nnx.Linear(2, 2, rngs=rngs)

    def __call__(self, x):
        x = self.fc1(x)
        x = self.fc2(x)
        return x

# Instantiating the Flax model
flax_custom_model = FlaxModel(rngs)

# Applying DropConnect to the Flax model
p = 0.5
model = dropconnect(cast(Any, flax_custom_model), p)

model

FlaxModel( # Param: 12 (48 B)
  fc1=Linear( # Param: 6 (24 B)
    bias=Param( # 2 (8 B)
      value=Array(shape=(2,), dtype=dtype('float32'))
    ),
    bias_init=<function zeros at 0x139be5080>,
    dot_general=<function dot_general at 0x1393c7a60>,
    dtype=None,
    in_features=2,
    kernel=Param( # 4 (16 B)
      value=Array(shape=(2, 2), dtype=dtype('float32'))
    ),
    kernel_init=<function variance_scaling.<locals>.init at 0x13b3d20c0>,
    out_features=2,
    param_dtype=float32,
    precision=None,
    preferred_element_type=None,
    promote_dtype=<function promote_dtype at 0x13b3d22a0>,
    use_bias=True
  ),
  fc2=DropConnectLinear( # Param: 6 (24 B)
    p=0.5,
    kernel=Param( # 4 (16 B)
      value=Array(shape=(2, 2), dtype=dtype('float32'))
    ),
    bias=Param( # 2 (8 B)
      value=Array(shape=(2,), dtype=dtype('float32'))
    ),
    in_features=2,
    out_features=2
  )
)

The variable p is the dropout probability for the weights.
The variable flax_custom_model is the model to be processed by DropConnect.
This model needs to be cast because custom models are not defined in the type checker later, but it is still valid and acceptable.